# Signals, wire types and interfaces

A signal is an abstracted entity encapsulating a virtual wire of a specific type. Whenever the signal changes, an event is registered and the simulation or the hardware will react accordingly.
The associated wire again can be of several types: Either a high/low (True/False) signal, it can represent a tri state signal with an extra high impedance state, a fractional arithmetic value with defined or undefined width, etc.

When a signal is connected to an interface to a sub circuit, it represents a virtual connector type on both ends. In most cases, it is desired to verify that the connection is made with the correct wire types. This is handled by the actual interface description which corresponds to a typical function call, with a few extra type hints.

A standard signal is created as follows:

In [2]:
from cyhdl import *

a = Signal(intbv()[5:])

The given argument is of wire type `intbv()` which behaves like a integer bit vector like a VHDL `unsigned`. It has hoever no notion of undefined values and can only take '0' or '1'. This is the default for most internal, well defined digital signals.

Print out a few properties:

In [3]:
a.name, a.resolve_type(), a.size()

('s_c209', myirl.emulation.myhdl_intbv.intbv, 5)

We note that the signal has an autogenerated name internal name. When created within a hardware unit (`@block`), it will in most cases during analysis receive its name from the local namespace.

If an explicit signal name should be displayed, for instance in interactive mode, a `name` parameter can be specified upon creation:

In [4]:
s0 = Signal(intbv()[64:], name = 's0_64bit')
s0

<s0_64bit>

### Signal initializers

In a sane hardware design, only the reset signal should be initialized with a default init value. All other signals are then initialized by this reset signal, typically. Thus, a given wire value is actually taken by the signal upon explicit reset.

However, there are exceptions:

* A clock signal must also start up with a well defined value
* A read only memory also contains preinitialized values that may be valid at start up
* In a simulation context, signals may also be preinitialized with a startup value

Within a test bench, a Signal can explicitely initialized **at start up** as follows:

In [5]:
a.init = True

Derived signals may prohibit this, if specific design rules are in place.

However note that this construct is only valid in a test bench or code that is explicitely output to a HDL language such as Verilog or VHDL.

## The interface

Let us define a simple unit with an input, output and an enable signal. 
We would like to keep the unit a little generic, being able to work with signal widths, except for the `en` signal which should be a boolean True/False only. We first define this Bool signal type ad-hoc using a `TypeGenerator` specification via the `.Type` notation:

In [6]:
Bool = Signal.Type(bool)

In [7]:
@block
def unit(a : Signal, en: Bool, q : Signal.Output):
    b = a.clone()
    
    logic = [
        b.set(~a)
    ]
    
    @always(en)
    def worker():
        if en:
            q.next = a
        else:
            q.next = b
    
    return instances()

**Caveat:** When using an ad-hoc type definition, keep in mind that library code might use a definition from another library (other than [myirl.library.basictypes](../../myirl/library/basictypes.py)). If interface type incompatibilies occur, you may have to import that same definition or derive from it. 

Also keep in mind, that `@container` signal classes (alias [port or bus records](ports.ipynb)) may have special type properties.

We now create an instance of this unit with 7 bit wide signals and output to VHDL (because of its strict interface):

In [8]:
a, b = [ Signal(intbv()[7:]) for _ in range(2) ]
en = Bool()

u = unit(a, en, b)

files = u.elab(targets.VHDL)

 Writing 'unit' to file /tmp/myirl_unit_9qzs4wyi/unit.vhdl 


In [9]:
!cat {files[0]}

-- File generated from source:
--     /home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/emulation/myhdl.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity unit is
    port (
        a : in unsigned(6 downto 0);
        en : in std_ulogic;
        q : out unsigned(6 downto 0)
    );
end entity unit;

architecture myhdl_emulation of unit is
    -- Local type declarations
    -- Signal declarations
    signal b : unsigned(6 downto 0);
begin
    
worker:
    process(en)
    begin
        if (en = '1') then
            q <= a;
        else
            q <= b;
        end if;
    end process;

    b <= not a;
end architecture myhdl_emulation;



### Interface inference rules

All `Signal` types passed as argument to a `@block` function infer to an interface object in the target. However, there are also so called 'pass through' arguments that are passed through during execution only, meaning, they don't infer to an interface entity in the target language.

Non-Signal types such as `bool`, `int` and other base types are considered pass-through, *unless* they are specifically put into the parameter declaration following a `*`.
When passed through, they are evaluated as a static constant during target generation and are eventually optimized away. This means they are suitable for conditional compilation, however it is considered better design practise to use them outside generators only. For example:

In [10]:
@block
def conditional_unit(a : Signal, b : Signal.Output, MODE : bool):
    
    if MODE:
        @always(a)
        def worker():
            a.next = ~b
    else:
        @always(a)
        def worker():
            a.next = b
    
    return instances()

We create instance two units with different configurations:

In [19]:
a, b = [ Signal(intbv()[5:]) for _ in range(2) ]
u1 = conditional_unit(a, b, False)
u2 = conditional_unit(a, b, True)

 Module conditional_unit: Existing instance conditional_unit, rename to conditional_unit_3 
 Module conditional_unit: Existing instance conditional_unit, rename to conditional_unit_4 


During a Python session (which may be interactive as a Jupyter Notebook), the `@block` objects register their implementations. Each creates a specific interface signature. Pass-Through parameters that may have a configuration character evaluate also statically in the interface signature. Thus, during hierarchical output to a target, several variants of a `block` implementation are emitted as necessary.

In [20]:
conditional_unit.implementations

['conditional_unitu_5u_5_0',
 'conditional_unitu_5u_5_0',
 'conditional_unitu_5u_5_1',
 'conditional_unitu_5u_5_0',
 'conditional_unitu_5u_5_1']

If we inspect the I/O port dictionary of an instance, we see that it has a `None` port descriptor, meaning, it's passed through as value.

In [17]:
u1.interface.portdict

OrderedDict([('a', (<IN : 5>, <s_4021>)),
             ('b', (<OUT : 5>, <s_fb97>)),
             ('MODE', (None, False))])

### Explicit pass-through

In some cases you may get a warning during inference, that a parameter is considered pass-through. To eliminate the warning, the current guideline is to explicitely annotate an argument as `PassThrough(type)` when it should not be inferred.

In [11]:
from cyhdl import *

@block
def unit0(msg : PassThrough(str),
          b : Signal.Output,
          *, ARGUMENT : str = "dynamic"):
    print(msg)
    wires = [ b.set(True) ]
    return instances()

When elaborated as VHDL, the `msg` parameter will be executed in the *native* executed body of the `@block`. The `ARGUMENT` parameters however will be inferred to a generic.

In [14]:
b = Signal(bool())
u = unit0("hello", b, "static")
f = u.elab(targets.VHDL)

 use default parameter ARGUMENT : dynamic 
 Module unit0: Existing instance unit0, rename to unit0_1 
hello
 Writing 'unit0_1' to file /tmp/myirl_unit0__ym4ckn3/unit0_1.vhdl 


In [15]:
!cat {f[0]}

-- File generated from source:
--     /tmp/ipykernel_21589/3230412913.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity unit0_1 is
    generic (
        ARGUMENT: string := "dynamic"
    );
    port (
        b : out std_ulogic
    );
end entity unit0_1;

architecture cyriteHDL of unit0_1 is
    -- Local type declarations
    -- Signal declarations
begin
    b <= '1';
end architecture cyriteHDL;



### Lazy interfacing

It is recommended to use strict typing whenever possible. When several choices are possible, use the tuple notation for the type hint (see also [Internals to strong typing](../notebooks/typechecking.ipynb). When you do checks for data types, you must do them outside the generator sub-functions.|

## Signal primitives

Apart from the generic `Signal` type, there are more specific types that are functionally compatible to myHDL.

### ClkSignal

The ClkSignal type is automatically initialized within a simulation testbench and is typically toggled by an `@always(delay(n))` process. For clock event synchronous processing, such signals are always required.

In [9]:
from inspect import signature
print(signature(ClkSignal))

(name=None, initval=False)


**Note:** ClkSignal is not myhdl-backwards-compatible

### ResetSignal

The ResetSignal is also mandatory when complex circuits are to be designed with either asynchronous or synchronous behaviour.

In [10]:
print(signature(ResetSignal))

(init, active, isasync=False, name=None)


### Signal example

The following unit represents a flip flop with either asynchronous or synchronous reset capabilities, depending on the `ResetSignal` configuration. We note that the description below does not denote any reset behaviour.
Thus, an entire architecture is switched between async/sync using configuration of the reset signal in one place.

In [11]:
@block
def reset_ff(clk: ClkSignal,
             reset: ResetSignal,
             a : Signal,
             q : Signal.Output):

    b = Signal(intbv(-1)[a.size():])
    
    @always_seq(clk.posedge, reset)
    def worker():
        b.next = a
        
    # Procedural non-myhdl wiring:   
    wires = [
        q.wireup(a)
    ]
    
    return instances()

First, we use this the synchronous way:

In [12]:
r = ResetSignal(init = False, active = True)
clk = ClkSignal()

u_sync = reset_ff(clk, r, a, b)

In [13]:
files = u_sync.elab(targets.Verilog)

 Writing 'reset_ff' to file /tmp/myirl_reset_ff_5kl3rz7m/reset_ff.v 
DEBUG Fallback wire for clk_c17c
DEBUG Fallback wire for s_8133


In [14]:
!grep  always {files[0]} -n5

16-    );
17-    // Local type declarations
18-    // Signal declarations
19-    reg [6:0] b;
20-    
21:    always @ (posedge clk ) begin : WORKER
22-        if (reset == 1'b1) begin
23-            b <= 7'b1111111; /* default */
24-        end else begin
25-            b <= a;
26-        end


### Asynchronous flip flop instance

Now we instance the asynchronous variant:

In [15]:
r = ResetSignal(init = True, active = False, isasync = True)
clk = ClkSignal()

u_async = reset_ff(clk, r, a, b)

 Module reset_ff: Existing instance reset_ff, rename to reset_ff_1 


In [16]:
files = u_async.elab(targets.VHDL)
!grep  process {files[0]} -n5

 Writing 'reset_ff_1' to file /tmp/myirl_reset_ff_po_0160_/reset_ff_1.vhdl 
26-    -- Signal declarations
27-    signal b : unsigned(6 downto 0);
28-begin
29-    
30-worker:
31:    process(clk, reset)
32-    begin
33-        if reset = '0' then
34-            b <= "1111111";
35-        elsif rising_edge(clk) then
36-            b <= a;
37-        end if;
38:    end process;
39-    q <= a;
40-end architecture myhdl_emulation;
41-


## Delayed signals

In simulation specific cases, signal delays may be modelled. However, signal delays are no longer specified on a signal level. To use simple signal delays, you can use an `@always(delay(x))` construct which internally infers to a `bulk_delay` generator. However, this is a non-portable construct and may yield deviating results on various targets. See [Simulation](simulation.ipynb) for more details.